In [3]:
import pandas as pd
import os,sys, imp
sys.path.append('./src/')
import doc_utils_clean as doc_utils
import feature_utils, common_utils
import defines
import numpy as np
from sklearn_crfsuite import scorers, CRF
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.model_selection import LeaveOneGroupOut,LeavePGroupsOut,GroupKFold

from operator import itemgetter
from sklearn_crfsuite.utils import flatten
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
import random
import model_utils
import seaborn as sns
import matplotlib.pyplot as plt
import json
from collections import Counter
import scipy
import nltk

import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.pipeline import Pipeline


In [4]:
dir_name="duplicate"

In [5]:
db_path = os.path.join(os.getcwd(),defines.PATH_TO_DFS,dir_name)
db_path

'/scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate'

In [6]:
merged_db = common_utils.concat_dbs(dir_name,"merged_db")
merged_db

,doc_idx,merged_idx,is_question,is_nar,par_idx_in_doc,par_pos_in_doc,sent_len,is_client,sent_idx_in_par,sent_pos_in_par,...,f_per_2,f_per_3,f_per_A,f_tense_BEINONI,f_tense_FUTURE,f_tense_IMPERATIVE,f_tense_PAST,POSTAG_NEG,POSTAG_yyCM,POSTAG_TTL
0,1,0,0.000000,0.000000,0.000000,0.004878,50.000000,0,0,1.000000,...,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
1,1,1,0.000000,0.000000,1.000000,0.009756,2.000000,1,0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
2,1,2,1.000000,0.000000,2.000000,0.014634,16.000000,0,0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
3,1,3,0.000000,0.000000,3.000000,0.019512,8.000000,1,0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
4,1,4,0.000000,0.000000,4.000000,0.024390,19.000000,0,0,1.000000,...,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38429,80,525,0.000000,0.000000,222.000000,0.991111,21.000000,1,3,0.666667,...,0.000000,0.200000,0.200000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
38430,80,526,0.000000,0.000000,222.000000,0.991111,27.000000,1,4,0.833333,...,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
38431,80,527,0.000000,0.000000,222.000000,0.991111,23.000000,1,5,1.000000,...,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
38432,80,528,1.000000,0.000000,223.000000,0.995556,19.000000,0,0,1.000000,...,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN


In [7]:
for col in merged_db.columns:
    if 'is' in col or 'idx' in col:
        merged_db[col] = merged_db[col].astype(int)

In [8]:
merged_db.fillna(value=0,inplace=True)

In [9]:
merged_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38434 entries, 0 to 38433
Data columns (total 66 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   doc_idx             38434 non-null  int64  
 1   merged_idx          38434 non-null  int64  
 2   is_question         38434 non-null  int64  
 3   is_nar              38434 non-null  int64  
 4   par_idx_in_doc      38434 non-null  int64  
 5   par_pos_in_doc      38434 non-null  float64
 6   sent_len            38434 non-null  float64
 7   is_client           38434 non-null  int64  
 8   sent_idx_in_par     38434 non-null  int64  
 9   sent_pos_in_par     38434 non-null  float64
 10  sent_pos_in_doc     38434 non-null  float64
 11  TOKEN               38434 non-null  int64  
 12  POSTAG_ADVERB       38434 non-null  float64
 13  POSTAG_AT           38434 non-null  float64
 14  POSTAG_BN           38434 non-null  float64
 15  POSTAG_BNT          38434 non-null  float64
 16  POST

## Scale sent_len, TOKEN

In [10]:
cols_to_scale=['sent_len','TOKEN']
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()
def scale(X):
    return scaler.fit_transform(X)

In [11]:
merged_db[cols_to_scale] = merged_db.groupby('doc_idx')[cols_to_scale].transform(lambda x: scaler.fit_transform(x.values[:,np.newaxis]).ravel())

In [14]:
merged_db

,doc_idx,merged_idx,is_question,is_nar,par_idx_in_doc,par_pos_in_doc,sent_len,is_client,sent_idx_in_par,sent_pos_in_par,...,f_per_2,f_per_3,f_per_A,f_tense_BEINONI,f_tense_FUTURE,f_tense_IMPERATIVE,f_tense_PAST,POSTAG_NEG,POSTAG_yyCM,POSTAG_TTL
0,1,0,0,0,0,0.004878,0.172043,0,0,1.000000,...,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1,0,0,1,0.009756,0.000000,1,0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1,2,1,0,2,0.014634,0.050179,0,0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1,3,0,0,3,0.019512,0.021505,1,0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1,4,0,0,4,0.024390,0.060932,0,0,1.000000,...,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38429,80,525,0,0,222,0.991111,0.093458,1,3,0.666667,...,0.000000,0.200000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38430,80,526,0,0,222,0.991111,0.121495,1,4,0.833333,...,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38431,80,527,0,0,222,0.991111,0.102804,1,5,1.000000,...,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38432,80,528,1,0,223,0.995556,0.084112,0,0,1.000000,...,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()
scaled_merge=pd.DataFrame(scaler.fit_transform(merged_db[cols_to_scale]),columns=cols_to_scale,index=merged_db.index)
scaled_merge

,sent_len,TOKEN
0,0.029518,0.036364
1,0.000602,0.000000
2,0.009036,0.006061
3,0.004217,0.006061
4,0.010843,0.024242
...,...,...
38776,0.012048,0.024242
38777,0.015663,0.030303
38778,0.013253,0.030303
38779,0.010843,0.012121


In [16]:
merged_db[cols_to_scale]=scaled_merge[cols_to_scale]
merged_db

,doc_idx,merged_idx,is_question,is_nar,par_idx_in_doc,par_pos_in_doc,sent_len,is_client,sent_idx_in_par,sent_pos_in_par,...,f_per_2,f_per_3,f_per_A,f_tense_BEINONI,f_tense_FUTURE,f_tense_IMPERATIVE,f_tense_PAST,POSTAG_NEG,POSTAG_yyCM,POSTAG_TTL
0,1,0,0,0,0,0.004878,0.029518,0,0,1.000000,...,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1,0,0,1,0.009756,0.000602,1,0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1,2,1,0,2,0.014634,0.009036,0,0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1,3,0,0,3,0.019512,0.004217,1,0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1,4,0,0,4,0.024390,0.010843,0,0,1.000000,...,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38776,80,525,0,0,222,0.991111,0.012048,1,3,0.666667,...,0.000000,0.200000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38777,80,526,0,0,222,0.991111,0.015663,1,4,0.833333,...,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38778,80,527,0,0,222,0.991111,0.013253,1,5,1.000000,...,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38779,80,528,1,0,223,0.995556,0.010843,0,0,1.000000,...,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Scale binary column

### Find correct value to replace 1

In [15]:
binary_columns = []
idx_columns = []
for col in merged_db.columns:
    if 'is' in col:
        binary_columns.append(col)
    if 'idx' in col:
        idx_columns.append(col)
non_pos_columns = binary_columns + idx_columns
non_pos_columns

['is_question',
 'is_nar',
 'is_client',
 'doc_idx',
 'merged_idx',
 'par_idx_in_doc',
 'sent_idx_in_par']

In [16]:
stat=merged_db[merged_db.columns[~merged_db.columns.isin(non_pos_columns)]].describe()
stat_mean=stat.mean(axis=1)
stat_mean

count   38434.000000
mean        0.092817
std         0.100241
min         0.000244
25%         0.033384
50%         0.066064
75%         0.124598
max         1.530226
dtype: float64

In [17]:
binary_lablel=np.array([stat_mean['mean'],stat_mean['50%']]).mean()
binary_lablel

0.07944037888517783

In [18]:
binary_lablel=0.5

In [19]:
for col in binary_columns:
    merged_db.replace({col: {1: binary_lablel}},inplace=True)

In [20]:
merged_db

,doc_idx,merged_idx,is_question,is_nar,par_idx_in_doc,par_pos_in_doc,sent_len,is_client,sent_idx_in_par,sent_pos_in_par,...,f_per_2,f_per_3,f_per_A,f_tense_BEINONI,f_tense_FUTURE,f_tense_IMPERATIVE,f_tense_PAST,POSTAG_NEG,POSTAG_yyCM,POSTAG_TTL
0,1,0,0.000000,0.000000,0,0.004878,0.172043,0.000000,0,1.000000,...,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1,0.000000,0.000000,1,0.009756,0.000000,0.500000,0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1,2,0.500000,0.000000,2,0.014634,0.050179,0.000000,0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1,3,0.000000,0.000000,3,0.019512,0.021505,0.500000,0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1,4,0.000000,0.000000,4,0.024390,0.060932,0.000000,0,1.000000,...,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38429,80,525,0.000000,0.000000,222,0.991111,0.093458,0.500000,3,0.666667,...,0.000000,0.200000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38430,80,526,0.000000,0.000000,222,0.991111,0.121495,0.500000,4,0.833333,...,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38431,80,527,0.000000,0.000000,222,0.991111,0.102804,0.500000,5,1.000000,...,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38432,80,528,0.500000,0.000000,223,0.995556,0.084112,0.000000,0,1.000000,...,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
scaled_name="scaled.per_doc{}".format(binary_lablel,lstrip='0')
scaled_name

'scaled.per_doc0.5'

## Save scaled merged_db

In [23]:
db_path

'/scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate'

In [22]:
for doc in merged_db['doc_idx'].unique():
    path=os.path.join(db_path,'{:02d}_{}.csv'.format(doc,scaled_name))
    df_to_write = merged_db[merged_db['doc_idx']==doc].iloc[:,2:]
    df_to_write.to_csv(path, index=False)